In [17]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import img_to_array, load_img

# Define paths
dataset_dir = r'E:\soil_clasification\Soil types'
train_dir = os.path.join(dataset_dir, r"E:\soil_clasification\Soil types/train")
valid_dir = os.path.join(dataset_dir, r"E:\soil_clasification\Soil types/valid")

# Ensure dataset has "train" and "valid" directories
# If not, split them manually:
def create_train_valid_split(dataset_dir, split_ratio=0.8):
    for category in ["unhealthy", "Healthy"]:
        category_path = os.path.join(dataset_dir, category)
        images = os.listdir(category_path)
        split_idx = int(len(images) * split_ratio)
        train_images = images[:split_idx]
        valid_images = images[split_idx:]

        # Create subdirectories for train/valid split
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(valid_dir, category), exist_ok=True)

        # Move files to respective folders
        for img in train_images:
            os.rename(
                os.path.join(category_path, img),
                os.path.join(train_dir, category, img)
            )
        for img in valid_images:
            os.rename(
                os.path.join(category_path, img),
                os.path.join(valid_dir, category, img)
            )

# Create train/valid split (run this only once)
if not os.path.exists(train_dir) or not os.path.exists(valid_dir):
    print("Creating train/valid split...")
    create_train_valid_split(dataset_dir)

# Filter corrupted images
def filter_corrupted_images(directory):
    for folder_name in os.listdir(directory):
        folder_path = os.path.join(directory, folder_name)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            try:
                img = load_img(file_path)
                _ = img_to_array(img)
            except Exception as e:
                print(f"Removing corrupted image: {file_path}")
                os.remove(file_path)

print("Filtering corrupted images...")
filter_corrupted_images(train_dir)
filter_corrupted_images(valid_dir)

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load data
batch_size = 32
img_size = (150, 150)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
epochs = 20
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=valid_data,
    callbacks=[early_stopping],
)

# Save the model
model.save("soil_texture_cnn_model.h5")

print("Model training completed and saved as 'soil_texture_cnn_model.h5'.")

# Evaluate the model
loss, accuracy = model.evaluate(valid_data)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


Creating train/valid split...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\soil_clasification\\Soil types\\unhealthy'